In [ ]:
# Discard the output of this cell.
#%%capture

# Install the required libraries.
!pip install tensorflow opencv-contrib-python youtube-dl moviepy pydot
# !pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the required libraries.
# !pip install tensorflow
import os
import cv2
# import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model


In [ ]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [ ]:

plt.figure(figsize=(20, 20))

all_classes_names = os.listdir("/content/drive/MyDrive/isl_data50")
sample_size = min(20, len(all_classes_names))
random_range = random.sample(all_classes_names, sample_size)

for counter, selected_class_name in enumerate(random_range, 1):
    video_files_names_list = os.listdir(f'/content/drive/MyDrive/isl_data50/{selected_class_name}')
    selected_video_file_name = random.choice(video_files_names_list)
    video_reader = cv2.VideoCapture(f'/content/drive/MyDrive/isl_data50/{selected_class_name}/{selected_video_file_name}')
    _, bgr_frame = video_reader.read()
    video_reader.release()
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)
    cv2.putText(rgb_frame, selected_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2)
    plt.subplot(5, 4, counter)
    plt.imshow(rgb_frame)
    plt.axis('off')

plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64,64
SEQUENCE_LENGTH = 50
DATASET_DIR = "/content/drive/MyDrive/isl_data50"
# CLASSES_LIST = ["hot_Cropped","lose_Cropped","doctor_Cropped", "accident_Cropped","help_Cropped","call_Cropped","thief_Cropped","pain_Cropped"]
CLASSES_LIST = []
folders = [f for f in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, f))]
for folder in folders:
    CLASSES_LIST.append(folder)

In [ ]:
def frame_extraction(video_path):
    frame_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frame_window = max(int(video_frame_count/SEQUENCE_LENGTH),1)
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter*skip_frame_window)
        success,frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
        normalized_frame = resized_frame/255
        frame_list.append(normalized_frame)
    video_reader.release()
    return frame_list

In [ ]:
def create_dataset():
    feature = []
    labels = []
    video_files_path = []
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR,class_name))
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR,class_name,file_name)
            frame = frame_extraction(video_file_path)
            if(len(frame) == SEQUENCE_LENGTH):
                feature.append(frame)
                labels.append(class_index)
                video_files_path.append(video_file_path)
    feature = np.asarray(feature)
    labels = np.array(labels)
    return feature,labels,video_file_path


In [ ]:
features,labels,video_file_paths = create_dataset()

Extracting Data of Class: White
Extracting Data of Class: Teacher
Extracting Data of Class: Summer
Extracting Data of Class: Window
Extracting Data of Class: you
Extracting Data of Class: Time
Extracting Data of Class: T-Shirt


In [ ]:
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
features_train,features_test,labels_train,labels_test = train_test_split(features,one_hot_encoded_labels,test_size=0.25,shuffle=True,random_state=seed_constant)

In [ ]:
from keras.layers import Conv2D, BatchNormalization, Bidirectional, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, ZeroPadding2D,ConvLSTM2D,LSTM,GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from keras.layers import UpSampling2D, Input, Concatenate,TimeDistributed, Add, GlobalMaxPooling2D, Multiply, Permute, multiply,MaxPooling2D, MaxPooling3D, Dropout, Flatten, GlobalAveragePooling3D, GlobalMaxPooling3D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau,EarlyStopping
from keras.metrics import Recall, Precision, Accuracy, MeanIoU
from keras import backend as K

In [ ]:
def create_convlstm_model():

    # We will use a Sequential model for model construction
    model = Sequential()
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (SEQUENCE_LENGTH,
                                                                                      IMAGE_HEIGHT, IMAGE_WIDTH, 3)))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 128, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))

    model.add(Flatten())
    model.add(Dense(len(CLASSES_LIST), activation = "softmax"))
    model.summary()
    return model

In [ ]:
# without attention

convlstm_model = create_convlstm_model()
print("Model Created Successfully")

In [ ]:
# without attention
plot_model(convlstm_model, to_file = r"C:\Users\ASUS\Downloads\experiment\Conv-lstm\convlstm.png",show_shapes= True,show_layer_names=True)

In [ ]:
# Create an Instance of Early Stopping Callback
# without attention
import keras
from keras import layers
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics values to the model
# opt = keras.optimizers.Adam(learning_rate=0.01)
opt = keras.optimizers.Nadam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    name="nadam",

)
convlstm_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])

# Start training the model. --> vary bacth size and epochs
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 500, batch_size = 8,
                                                     shuffle = True, validation_split = 0.2,
                                                     callbacks = [early_stopping_callback])

In [ ]:
model_evaluation_history = convlstm_model.evaluate(features_test,labels_test)

In [ ]:
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

model_file_name = fr'C:\Users\ASUS\Downloads\experiment\Conv-lstm\convlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
convlstm_model.save(model_file_name)

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name, to_file = None):

    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))
    if to_file:
        plt.savefig(to_file)

    # Add legend to the plot.
    plt.legend()

In [ ]:
# Visualize the training and validation loss metrices.
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss', to_file = r"C:\Users\ASUS\Downloads\experiment\Conv-lstm\loss.png")

In [ ]:
# Visualize the training and validation accuracy metrices.
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy', to_file = r"C:\Users\ASUS\Downloads\experiment\Conv-lstm\accuracy.png")